In [1]:
# This is a simulated Jupyter Notebook content focusing on Data Cleaning.
# You can copy and paste this into a .ipynb file and run it cell by cell.

# --- Markdown Cell: Introduction ---
# # Data Cleaning and Preprocessing Pipeline
#
# This notebook is dedicated to the essential steps of data cleaning and preprocessing for the job posting dataset.
# The goal is to transform the raw data into a clean, structured, and usable format for subsequent analysis.
#
# **Dataset:** The job posting data located at `C:\Users\DELL\Downloads\archive\job_descriptions.csv`.
#
# **Output:** A cleaned DataFrame saved as a `joblib` file (`processed_job_data.joblib`).
#
# Your request to demonstrate reloading the file after saving with `joblib` has been included.

# --- Code Cell: Import Libraries ---
# ## 1. Import Libraries
# Essential libraries required for data manipulation and serialization.
import pandas as pd
import json # To handle the Company Profile JSON string
import re # For regular expressions to parse salary and experience
import joblib # For saving and loading Python objects efficiently

# --- Markdown Cell: Data Loading ---
# ## 2. Data Loading
#
# We will load the job posting dataset into a pandas DataFrame from the specified CSV file.
#
# **Important:** Ensure that the `job_descriptions.csv` file is located at the provided path.

# --- Code Cell: Load Data ---
# Define the filename for the job descriptions data with the full path.
job_data_filename = "C:\\Users\\DELL\\Downloads\\archive\\job_descriptions.csv"

try:
    # Read the data from the CSV file into a DataFrame.
    df = pd.read_csv(job_data_filename)
    print(f"Successfully loaded data from '{job_data_filename}'.")
except FileNotFoundError:
    print(f"Error: The file '{job_data_filename}' was not found.")
    print("Please ensure the CSV file exists at the specified path.")
    # Create an empty DataFrame to prevent further errors if the file is not found.
    df = pd.DataFrame()
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    df = pd.DataFrame()


# Only proceed if the DataFrame is not empty after loading attempt.
if not df.empty:
    # Display the first few rows of the DataFrame to verify loading.
    print("DataFrame Head:")
    print(df.head())
    # Display concise summary of the DataFrame, including data types and non-null values.
    print("\nDataFrame Info:")
    print(df.info())
else:
    print("\nDataFrame is empty. Cannot proceed with data cleaning without data.")


# --- Markdown Cell: Data Cleaning and Preprocessing ---
# ## 3. Data Cleaning and Preprocessing
#
# This is the core section of this notebook, where raw data is transformed into a clean and usable format.
#
# **Steps Performed:**
# 1.  **Convert 'Job Posting Date' to `datetime`:** Ensures proper handling of dates for time-based filtering and analysis.
# 2.  **Parse 'Company Profile' JSON Strings:** Converts string representations of company profiles into structured Python dictionaries. This includes handling potential malformed JSON entries by returning an empty dictionary and logging errors.
# 3.  **Extract 'Company Sector':** Derives the 'Sector' information from the newly parsed 'Company Profile' dictionaries.
# 4.  **Extract 'Min Experience Years':** Parses the 'Experience' string (e.g., "5 to 15 Years") to extract the minimum years of experience as an integer.
# 5.  **Extract 'Min Salary USD':** Parses the 'Salary Range' string (e.g., "$50K-$100K") to extract the minimum salary in USD as an integer.
# 6.  **Handle Missing Values/Inconsistencies:** The parsing functions are designed to return `None` or empty dictionaries for missing/invalid entries, preventing errors in subsequent steps.

# --- Code Cell: Perform Data Cleaning and Preprocessing ---
if not df.empty:
    # Convert 'Job Posting Date' to datetime objects.
    df['Job Posting Date'] = pd.to_datetime(df['Job Posting Date'])
    print("\n'Job Posting Date' after conversion:")
    print(df['Job Posting Date'].head())
    print(df['Job Posting Date'].dtype)

    # Function to safely parse the 'Company Profile' string into a dictionary.
    def parse_company_profile(profile_str):
        if pd.isna(profile_str):
            return {}
        try:
            # Replace escaped double quotes (e.g., """") with single double quotes (") for valid JSON.
            cleaned_str = str(profile_str).replace('""""', '"')
            # Remove any outer double quotes that might have been added by CSV parsing.
            if cleaned_str.startswith('"') and cleaned_str.endswith('"'):
                cleaned_str = cleaned_str[1:-1]
            return json.loads(cleaned_str)
        except json.JSONDecodeError as e:
            # Print an error if parsing fails and return an empty dictionary.
            print(f"Error decoding JSON: {e} for string: {profile_str[:50]}...")
            return {} # Return an empty dict for errors

    # Apply the parsing function to create a new column with parsed company profiles.
    df['Company Profile_Parsed'] = df['Company Profile'].apply(parse_company_profile)

    print("\n'Company Profile_Parsed' column (first entry):")
    print(df['Company Profile_Parsed'].iloc[0])

    # Extract 'Sector' from the parsed company profile for potential future analysis.
    df['Company Sector'] = df['Company Profile_Parsed'].apply(lambda x: x.get('Sector'))
    print("\n'Company Sector' extracted:")
    print(df['Company Sector'].head())

    # Function to extract the minimum experience in years from the 'Experience' string.
    def extract_min_experience(experience_str):
        if pd.isna(experience_str): # Check for missing values
            return None
        # Use regex to find digits followed by 'to'
        match = re.search(r'(\d+)\s*to', str(experience_str))
        if match:
            return int(match.group(1))
        return None

    # Apply the function to create a new numerical 'Min Experience Years' column.
    df['Min Experience Years'] = df['Experience'].apply(extract_min_experience)
    print("\n'Min Experience Years' extracted:")
    print(df['Min Experience Years'].head())

    # Function to extract the minimum salary in USD (in thousands)
    def extract_min_salary(salary_str):
        if pd.isna(salary_str): # Check for missing values
            return None
        # Use regex to find a dollar sign, digits, and 'K' (for thousands).
        match = re.search(r'\$(\d+)K', str(salary_str))
        if match:
            return int(match.group(1)) * 1000 # Convert thousands (K) to actual value.
        return None

    # Apply the function to create a new numerical 'Min Salary USD' column.
    df['Min Salary USD'] = df['Salary Range'].apply(extract_min_salary)
    print("\n'Min Salary USD' extracted:")
    print(df['Min Salary USD'].head())

    # Display info of the DataFrame after cleaning to see new columns and types.
    print("\nDataFrame Info after Cleaning:")
    print(df.info())
else:
    print("\nSkipping Data Cleaning and Preprocessing as DataFrame is empty.")


# --- Markdown Cell: Saving Cleaned Data ---
# ## 4. Saving Cleaned Data
#
# After all cleaning and preprocessing steps are complete, the resulting DataFrame is saved to a `joblib` file. This allows you to easily load this cleaned version of the data directly in future analysis notebooks, saving time and computational resources by avoiding repeated preprocessing.
#
# **Output File:** `processed_job_data.joblib`

# --- Code Cell: Save Cleaned Data ---
if not df.empty:
    # Define a filename for the processed DataFrame.
    processed_data_filename = 'processed_job_data.joblib'
    # Save the DataFrame to a file using joblib.
    joblib.dump(df, processed_data_filename)
    print(f"\nCleaned DataFrame successfully saved to '{processed_data_filename}'.")
else:
    print("\nSkipping saving cleaned data as DataFrame is empty.")

# --- Markdown Cell: Reloading Cleaned Data (Demonstration) ---
# ## 5. Reloading Cleaned Data
#
# This section demonstrates how to reload the `processed_job_data.joblib` file back into a new DataFrame. This is useful for continuing your analysis from the cleaned data without needing to re-run all the preprocessing steps.

# --- Code Cell: Reload Data with joblib ---
try:
    # Load the DataFrame from the joblib file.
    loaded_df = joblib.load(processed_data_filename)
    print(f"\nSuccessfully reloaded DataFrame from '{processed_data_filename}'.")
    print("\nLoaded DataFrame Head (from joblib):")
    print(loaded_df.head())
    print("\nLoaded DataFrame Info (from joblib):")
    print(loaded_df.info())
except FileNotFoundError:
    print(f"\nError: The file '{processed_data_filename}' was not found. Please ensure it was saved correctly.")
except Exception as e:
    print(f"\nAn error occurred while reloading the file: {e}")


# --- Markdown Cell: Conclusion ---
# ## 6. Conclusion
#
# This notebook successfully demonstrated the data cleaning and preprocessing pipeline for the job posting dataset. The raw data has been transformed into a structured format, with appropriate data types, parsed JSON fields, and extracted numerical features. The cleaned DataFrame is now ready for in-depth exploratory data analysis, visualization, and modeling in subsequent notebooks or scripts. We also demonstrated how to save and efficiently reload this cleaned data using `joblib`.


Successfully loaded data from 'C:\Users\DELL\Downloads\archive\job_descriptions.csv'.
DataFrame Head:
             Job Id     Experience Qualifications Salary Range    location  \
0  1089843540111562  5 to 15 Years         M.Tech    $59K-$99K     Douglas   
1   398454096642776  2 to 12 Years            BCA   $56K-$116K    Ashgabat   
2   481640072963533  0 to 12 Years            PhD   $61K-$104K       Macao   
3   688192671473044  4 to 11 Years            PhD    $65K-$91K  Porto-Novo   
4   117057806156508  1 to 12 Years            MBA    $64K-$87K    Santiago   

            Country  latitude  longitude  Work Type  Company Size  ...  \
0       Isle of Man   54.2361    -4.5481     Intern         26801  ...   
1      Turkmenistan   38.9697    59.5563     Intern        100340  ...   
2  Macao SAR, China   22.1987   113.5439  Temporary         84525  ...   
3             Benin    9.3077     2.3158  Full-Time        129896  ...   
4             Chile  -35.6751   -71.5429     Intern        